In [17]:
sc.version

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'3.0.1-amzn-0'

In [4]:
#Set up environment

%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1715403272843_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1715403272843_0003,pyspark,idle,Link,Link,✔


In [5]:
#Check existing packages
sc.list_packages()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version   
-------------------------- ----------
beautifulsoup4             4.9.3     
boto                       2.49.0    
click                      7.1.2     
jmespath                   0.10.0    
joblib                     0.17.0    
lxml                       4.6.1     
mysqlclient                1.4.2     
nltk                       3.5       
nose                       1.3.4     
numpy                      1.16.5    
pip                        9.0.1     
py-dateutil                2.2       
python37-sagemaker-pyspark 1.4.1     
pytz                       2020.1    
PyYAML                     5.3.1     
regex                      2020.10.28
setuptools                 28.8.0    
six                        1.13.0    
tqdm                       4.51.0    
wheel                      0.29.0    
windmill                   1.6

In [11]:
#Install pandas to be able to work with dataframes
sc.install_pypi_package("pandas==1.0.5", "https://pypi.org/simple")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
data = spark.read.csv('s3://finalproject-nat-s3/submissions.csv', 
                      header=True, 
                      inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#See structure of the dataframe 
print('Total Columns: %d' % len(data.dtypes))
print('Total Rows: %d' % data.count())
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Columns: 7
Total Rows: 11500261
root
 |-- id: string (nullable = true)
 |-- created: string (nullable = true)
 |-- author: string (nullable = true)
 |-- score: string (nullable = true)
 |-- title: string (nullable = true)
 |-- selftext: string (nullable = true)
 |-- num_comments: string (nullable = true)

In [16]:
#Convert to parquet for optimizing storage and speedup
data.to_parquet('%s/s3://finalproject-nat-s3/submissions.parquet' % path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DataFrame' object has no attribute 'to_parquet'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 1403, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
AttributeError: 'DataFrame' object has no attribute 'to_parquet'



In [20]:
#Visualize structure of values
data.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------------+-------------+-----+--------------------+---------+------------+
|   id|         created|       author|score|               title| selftext|num_comments|
+-----+----------------+-------------+-----+--------------------+---------+------------+
|8shht|2009-06-14 18:22|  u/[deleted]|    9|Introducing the R...|[deleted]|           1|
|8si6m|2009-06-14 20:42|  u/[deleted]|   18|Potential Reddit ...|[deleted]|          18|
|8sur3|2009-06-16 00:19|  u/[deleted]|    7|How do I get over...|[deleted]|           8|
|8sw2f|2009-06-16 03:30|u/notmyrealsn|    3|Long distance rel...|     null|           9|
|8swor|2009-06-16 05:01|        u/cr3|   10|Name your favouri...|     null|          20|
|8sx2g|2009-06-16 05:58|  u/[deleted]|    2|Dear Reddit, need...|[deleted]|           7|
|8t0lu|2009-06-16 12:13|u/rogerssucks|    1|"How many of you ...|     null|           3|
|8t1b6|2009-06-16 13:22|  u/[deleted]|    5|I kept up contact...|[deleted]|           5|
|8t1rm|2009-06-16 14:

In [25]:
#For context, selftext refers to the body text of the submission

#We can see that there are a lot of deleted entries. I also know there are "[removed]" values as well.
#It is necessary to clean the following:
#[deleted], [removed], empty/null

filtered_data = data.filter(data['selftext'] != "[deleted]")
filtered_data = filtered_data.filter(filtered_data['selftext'] != "[removed]")


filtered_data.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  id|             created|              author|               score|               title|            selftext|        num_comments|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               8tw9m|    2009-06-19 08:33|         u/[deleted]|                   7|How do you move p...|I am currently go...|                null|
|The fact that our...| but it is probab...| when I lie in be...|             snoring| as he did during...| the sensation of...|                  19|
|               8u1vm|    2009-06-19 20:34|         u/[deleted]|                   4|About to start a ...|"My girlfriend an...| however our sche...|
|         Beyond that|                well| it's up the air....| part of me sees ...|                 yes|

In [27]:
#Looks messy, so I'll take a look just at the selftext column

filtered_data[['selftext']].show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|            selftext|
+--------------------+
|I am currently go...|
| the sensation of...|
|"My girlfriend an...|
| but I look at he...|
| if anyone has an...|
+--------------------+
only showing top 5 rows

In [ ]:
#Sources
#https://spark.apache.org/docs/latest/sql-data-sources-csv.html
